This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/data_000637.txt',nrows=15000) #this gives N rows
data_full = pd.read_csv('/content/data_000637.txt')
data.head()
df = pd.DataFrame(data)
df_full = pd.DataFrame(data_full)
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
0         1     0          123  3869200167        2374        26
1         1     0          124  3869200167        2374        27
2         1     0           63  3869200167        2553        28
3         1     0           64  3869200167        2558        19
4         1     0           64  3869200167        2760        25
...     ...   ...          ...         ...         ...       ...
14995     1     1            4  3869200316        3399         9
14996     1     1           17  3869200316        3400        15
14997     1     1           10  3869200316        3530        16
14998     1     1            8  3869200316        3533        18
14999     1     0          139  3869200316        3539         0

[15000 rows x 6 columns]


2. Estimate the number of BX in a ORBIT (the value x).

Hint: check when the BX counter reaches the maximum value before being reset to 0.

In [4]:
BX_max_value = df_full.BX_COUNTER.max()
print("Max BX number in a orbit: "+str(BX_max_value))

Max BX number in a orbit: 3563


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [5]:
df['TIME_ns'] = df.apply(lambda ns: ns.TDC_CHANNEL + ns.BX_COUNTER + ns.TDC_MEAS, axis = 1)
df['TIME_ns'] = pd.to_datetime(df['TIME_ns'])
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
14995     1     1            4  3869200316        3399         9   
14996     1     1           17  3869200316        3400        15   
14997     1     1           10  3869200316        3530        16   
14998     1     1            8  3869200316        3533        18   
14999     1     0          139  3869200316        3539         0   

                            TIME_ns  
0     1970-01-01 00:00:00.000002523  
1     1970-01-01 00:00:00.000002525  
2     1970-01-01 00:00:00.000002644  
3     1970-01-01 00:00:00.00000

4. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset

In [6]:
df['DURATION'] = df['TIME_ns'].dt.time
print(df)

       HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS  \
0         1     0          123  3869200167        2374        26   
1         1     0          124  3869200167        2374        27   
2         1     0           63  3869200167        2553        28   
3         1     0           64  3869200167        2558        19   
4         1     0           64  3869200167        2760        25   
...     ...   ...          ...         ...         ...       ...   
14995     1     1            4  3869200316        3399         9   
14996     1     1           17  3869200316        3400        15   
14997     1     1           10  3869200316        3530        16   
14998     1     1            8  3869200316        3533        18   
14999     1     0          139  3869200316        3539         0   

                            TIME_ns         DURATION  
0     1970-01-01 00:00:00.000002523  00:00:00.000002  
1     1970-01-01 00:00:00.000002525  00:00:00.000002  
2     1970-01-01 0

In [ ]:
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIMENS,TOTAL_TIME(sec)
0,1,0,123,3869200167,2374,26,5.937167e+04,0.000059
1,1,0,124,3869200167,2374,27,1.187442e+05,0.000059
2,1,0,63,3869200167,2553,28,1.825925e+05,0.000064
3,1,0,64,3869200167,2558,19,2.465583e+05,0.000064
4,1,0,64,3869200167,2760,25,3.155792e+05,0.000069
...,...,...,...,...,...,...,...,...
14995,1,1,4,3869200316,3399,9,6.764781e+08,0.000085
14996,1,1,17,3869200316,3400,15,6.765631e+08,0.000085
14997,1,1,10,3869200316,3530,16,6.766513e+08,0.000088
14998,1,1,8,3869200316,3533,18,6.767397e+08,0.000088


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [8]:
noisy_channels = df.groupby(['TDC_CHANNEL'])['BX_COUNTER']
noisy_channels = noisy_channels.max()
df1 = pd.DataFrame(noisy_channels.head(3))
print(df1)

             BX_COUNTER
TDC_CHANNEL            
1                  3558
2                  3555
3                  3557


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). 

In [9]:
non_empty = df.groupby(['ORBIT_CNT'])
non_empty = non_empty.count()
print(len(non_empty))

150


7. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [11]:
unique_orbits = df[df['TDC_CHANNEL'] == 139]['ORBIT_CNT'].unique()
print(len(unique_orbits))

150


8. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

In [12]:
FPGA0 = df[df['FPGA'] == 0]
FPGA0 = pd.Series(df[['TDC_CHANNEL']].value_counts())
print(FPGA0)

FPGA1 = df[df['FPGA'] == 1] 
FPGA1 = pd.Series(df[['TDC_CHANNEL']].value_counts())
print(FPGA1)

TDC_CHANNEL
139            1268
64              752
63              749
61              571
62              542
               ... 
107               6
106               6
55                6
125               6
67                5
Length: 129, dtype: int64
TDC_CHANNEL
139            1268
64              752
63              749
61              571
62              542
               ... 
107               6
106               6
55                6
125               6
67                5
Length: 129, dtype: int64
